## **1. Data cleaning**

**OVERVIEW**
- This workbook will focus on ensuring data downloaded from `data/raw/prices`is in the best quality for the following analyses
- It implies checking **missing values (NaNs), formatting, merging of tables, data transformation, etc.**
- All clean data will be saved in `data/processed`

**SUMMARY RESULTS**
- The resulting processed dataset (`asset_universe`) has **1,508 rows and 7 columns** (excluding the 'Date' column and 'IRX' risk-free rate proxy)
- The date range goes from **2019-01-02** to **2024-12-30** and there are **NO missing values** to deal with

#### **1.1 Importing necessary libraries**

In [1]:
import pandas as pd
import random
from src.helpers_io import raw_path, processed_path, read_csv_raw, save_csv_processed

# Creating path to 'data/prices'
raw_prices_dir = raw_path("prices")

#### **1.2 Loading datasets**

In [2]:
# Quick view of saved files and storing ticker names
tickers = []

for file in sorted(raw_prices_dir.glob("*.csv")):
    filename = file.name.split(sep="_")[0]
    tickers.append(filename)
    print(file.name)

EURUSD_prices.csv
GLD_prices.csv
IEF_prices.csv
IRX_prices.csv
SPY_prices.csv
UNG_prices.csv
USDJPY_prices.csv
USO_prices.csv


#### **1.3 Inspecting structure and data types**

Based on the results, all columns have **correct data types** and **non-missing values**

In [3]:
# Setting a seed
random.seed(123)

# Small sample of assets to check
assets = random.sample(tickers, k=3)

for asset in assets:
    data = read_csv_raw(f"{raw_prices_dir / asset}_prices.csv", parse_dates=["Date"])
    print(asset)
    display(data.head(5), data.info(), data.isna().any())

EURUSD
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1564 non-null   datetime64[ns]
 1   Close   1564 non-null   float64       
 2   High    1564 non-null   float64       
 3   Low     1564 non-null   float64       
 4   Open    1564 non-null   float64       
 5   Volume  1564 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 73.4 KB


,Date,Close,High,Low,Open,Volume
0,2019-01-01,1.149306,1.155001,1.146500,1.149425,0
1,2019-01-02,1.146171,1.149700,1.134572,1.146132,0
2,2019-01-03,1.131811,1.140914,1.131734,1.131734,0
3,2019-01-04,1.139108,1.141774,1.134816,1.139095,0
4,2019-01-07,1.141044,1.147447,1.140524,1.141292,0


None

Date      False
Close     False
High      False
Low       False
Open      False
Volume    False
dtype: bool

IEF
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1509 entries, 0 to 1508
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1509 non-null   datetime64[ns]
 1   Close   1509 non-null   float64       
 2   High    1509 non-null   float64       
 3   Low     1509 non-null   float64       
 4   Open    1509 non-null   float64       
 5   Volume  1509 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 70.9 KB


...

Date      False
Close     False
High      False
Low       False
Open      False
Volume    False
dtype: bool

USO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1509 entries, 0 to 1508
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1509 non-null   datetime64[ns]
 1   Close   1509 non-null   float64       
 2   High    1509 non-null   float64       
 3   Low     1509 non-null   float64       
 4   Open    1509 non-null   float64       
 5   Volume  1509 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 70.9 KB


,Date,Close,High,Low,Open,Volume
0,2019-01-02,78.800003,80.639999,74.879997,75.360001,4137850
1,2019-01-03,79.599998,80.080002,77.199997,79.919998,4027375
2,2019-01-04,81.440002,83.120003,80.480003,81.360001,4864513
3,2019-01-07,82.320000,84.160004,81.839996,82.160004,4031138
4,2019-01-08,84.000000,84.400002,82.879997,83.440002,3644150


None

Date      False
Close     False
High      False
Low       False
Open      False
Volume    False
dtype: bool

#### **1.4 Standardizing dataframes**

In [4]:
datasets = {}

for file in sorted(raw_prices_dir.glob("*.csv")):
    # Getting asset namem ONLY
    filename = file.name.split(sep="_")[0]

    # Reading CSV and converting 'Date' into datetime format
    data = read_csv_raw(f"prices/{file.name}", parse_dates=["Date"])
    data = data.set_index("Date").sort_index(ascending=True)

    # Keeping 'Date' and renaming 'Close'
    data = data.rename(columns={"Close": filename})
    data = data[[filename]]

    # Adding it to 'datasets' library
    datasets[filename] = data

#### **1.5 Analizing common date ranges for merging**

In [5]:
# Creating a new DataFrame
comparison_table = {"ticker": [], "start_date": [], "end_date": [], "n_rows": [], "nan_values": []}

# Iterating each dataset
for ticker, dataset in datasets.items():
    comparison_table["ticker"].append(ticker)
    comparison_table["start_date"].append(dataset.index.min())
    comparison_table["end_date"].append(dataset.index.max())
    comparison_table["n_rows"].append(len(dataset))
    comparison_table["nan_values"].append(dataset.isna().sum().iloc[0])

# Converting my dictionary into a DataFrame and sorting values based on 'start_date'
comparison_table = pd.DataFrame(comparison_table)
comparison_table.set_index("ticker", inplace=True)
comparison_table = comparison_table.sort_values("start_date", ascending=True)

# Defining merging window
max_start_date = max(comparison_table["start_date"])
min_end_date = min(comparison_table["end_date"])

print(f"""start_date: {max_start_date}
end_date: {min_end_date}""")
comparison_table

start_date: 2019-01-02 00:00:00
end_date: 2024-12-30 00:00:00


,start_date,end_date,n_rows,nan_values
ticker,,,,
EURUSD,2019-01-01,2024-12-30,1564,0
USDJPY,2019-01-01,2024-12-30,1564,0
GLD,2019-01-02,2024-12-30,1509,0
IEF,2019-01-02,2024-12-30,1509,0
IRX,2019-01-02,2024-12-30,1509,0
SPY,2019-01-02,2024-12-30,1509,0
UNG,2019-01-02,2024-12-30,1509,0
USO,2019-01-02,2024-12-30,1509,0


#### **1.6. Merging datasets**

In [6]:
# Slicing on common window
aligned = []
for ticker, df in datasets.items():
    aligned.append(df.loc[max_start_date:min_end_date])

# Concat per columns, inner join in dates (indexes)
asset_universe = pd.concat(aligned, axis=1, join="inner").sort_index()
asset_universe

,EURUSD,GLD,IEF,IRX,SPY,UNG,USDJPY,USO
Date,,,,,,,,
2019-01-02,1.146171,121.330002,88.567047,2.365,224.995316,97.959999,109.667999,78.800003
2019-01-03,1.131811,122.430000,89.271088,2.355,219.626282,96.839996,107.441002,79.599998
2019-01-04,1.139108,121.440002,88.550110,2.358,226.982834,100.360001,107.807999,81.440002
2019-01-07,1.141044,121.860001,88.304131,2.353,228.772537,97.519997,108.522003,82.320000
2019-01-08,1.147974,121.529999,88.083626,2.400,230.921967,98.959999,108.615997,84.000000
...,...,...,...,...,...,...,...,...
2024-12-23,1.043308,240.960007,88.481308,4.215,587.787476,15.630000,156.533005,73.059998
2024-12-24,1.040583,241.440002,88.529320,4.200,594.320740,16.139999,157.164993,73.650002
2024-12-26,1.039955,243.070007,88.596474,4.215,594.360413,15.370000,157.132996,73.129997


In [7]:
# Quick check of 'asset_universe'
print(asset_universe.info())    # Verifying data length and dtype
display(asset_universe.isna().sum())    # Veryfing NaNs

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1508 entries, 2019-01-02 to 2024-12-30
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EURUSD  1508 non-null   float64
 1   GLD     1508 non-null   float64
 2   IEF     1508 non-null   float64
 3   IRX     1508 non-null   float64
 4   SPY     1508 non-null   float64
 5   UNG     1508 non-null   float64
 6   USDJPY  1508 non-null   float64
 7   USO     1508 non-null   float64
dtypes: float64(8)
memory usage: 106.0 KB
None


EURUSD    0
GLD       0
IEF       0
IRX       0
SPY       0
UNG       0
USDJPY    0
USO       0
dtype: int64

#### **1.7 Exporting processed dataset**

In [8]:
# Saving CSV into data/processed
risk_free = asset_universe["IRX"].copy()
asset_universe = asset_universe.drop(columns=["IRX"])

try:
    save_csv_processed(risk_free, "risk_free.csv", index=True)
    save_csv_processed(asset_universe, "asset_universe.csv", index=True)

    asset_universe_path = processed_path("asset_universe.csv")
    risk_free_path = processed_path("risk_free.csv")

    if asset_universe_path.exists() and risk_free_path:
        print("Successfully exported! ✅")
    else:
        print("Export failed ⚠️")

except Exception as e:
    print(f"""Error during export ❌
        Details: {e}""")

Successfully exported! ✅
